In [1]:
import pandas as pd
train_df = pd.read_csv('poem_train.csv', encoding='utf-8')
test_df = pd.read_csv('poem_test.csv', encoding='utf-8')
train_df

,作者,詩名,內容
0,李白,菩薩蠻·平林漠漠煙如織,平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1,李白,把酒問月,青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2,李白,春思,燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3,李白,春夜洛城聞笛,誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4,李白,古風 其十九,西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
...,...,...,...
2726,白居易,彆元九後詠所懷,零落桐葉雨，蕭條槿花風。悠悠早秋意，生此幽閒中。況與故人彆，中懷正無悰。勿雲不相送，心到青門...
2727,白居易,早秋曲江感懷,離離暑雲散，嫋嫋涼風起。池上秋又來，荷花半成子。朱顏易銷歇，白日無窮已。人壽不如山，年光急於...
2728,白居易,東墟晚歇 時退居渭村。,涼風冷露蕭索天，黃蒿紫菊荒涼田。繞塚秋花少顏色，細蟲小蝶飛翻翻。中有騰騰獨行者，手拄漁竿不騎...
2729,白居易,南秦雪,往歲曾為西邑吏，慣從駱口到南秦。\r\n三時雲冷多飛雪，二月山寒少有春。\r\n我思舊事猶惆...


In [3]:
trans = {'李白':0,'杜甫':1,'白居易':2}
# pandas replace替換資料
y_train = train_df['作者'].replace(trans)
y_test = test_df['作者'].replace(trans)
y_train

0       0
1       0
2       0
3       0
4       0
       ..
2726    2
2727    2
2728    2
2729    2
2730    2
Name: 作者, Length: 2731, dtype: int64

In [ ]:
from jieba import cut
s = '平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。'
s = (s.replace("\r", ' ')
      .replace("\n", ' ')
      .replace("，", ' ')
      .replace("。", ' '))
' '.join(cut(s))

In [ ]:
# pandas apply
def poemcut(s):
    s = (s.replace("\r", ' ')
      .replace("\n", ' ')
#       .replace("，", ' ')
#       .replace("。", ' ')
        )
    return ' '.join(cut(s))

x_train = train_df['內容'].apply(poemcut)
x_test = test_df['內容'].apply(poemcut)


In [ ]:
# fit 找出有幾個欄位
# transform 轉換
# 訓練資料: fit + transform
# 測試資料: transform

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
# fit_transform 會處理標點符號
x_train_vec = vec.fit_transform(x_train)

In [ ]:
print(x_train_vec)
#Sparse Matrix 只存非0的值

In [ ]:
# 字典
reverse = {v:k for k,v in vec.vocabulary_.items()}
i = int(input('輸入欄位->詞'))
print('對應的詞:',[reverse[i]])

In [ ]:
# 測試資料不fit
x_test_vec = vec.transform(x_test)
print(x_test_vec)

In [ ]:
# 單純貝式分類器
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_vec)
accuracy_score(pre,y_test)

In [ ]:
# 混淆矩陣
from sklearn.metrics import confusion_matrix
labels = ['李白','杜甫','白居易']
pd.DataFrame(confusion_matrix(y_test,pre),
             columns=[f'{l} (預測)' for l in labels],
             index=[f'{l} (正確)' for l in labels])

In [ ]:
p = input('請寫一首詩')
p = poemcut(p)
p_vec = vec.transform([p])
print('應該是誰寫的->',labels[clf.predict(p_vec)[0]])